## <b>■ 텐서 플로우를 활용한 신경망 구성 복습</b>
    1. 텐서플로우 1.x 버전 기본문법
            그래프 생성 영역
            --------------
            그래프 실행 영역
            
    2. 텐서 플로우로 이용한 단층 신경망
    3. 텐서 플로우를 이용한 다층 신경망
    4. 텐서 플로우를 이용해 CNN을 이용한 신경망
        MNIST 데이터는 load_mnist 함수만을 이용해서 쉽게 데이터를 로드할 수 있게 구성했는데 
        현업에서 신경망을 구현할 때는 load_mnist와 같은 함수를 직접 생성해야하므로
        신경망으로 데이터를 로드하는 4개 함수를 생성
            1. image_load
            2. label_load
            3. next_batch
            4. shuffle_batch
            
### <b>■ 이파리 데이터를 로드할 수 있는 4개의 함수 생성</b>
    
    1. 질병 이파리 사진 확인
        darknamer.exe 프로그램을 이용해서 사진이름을 전부 숫자로 변경
            1 ~ 10000번으로 이름 변경
            1-9500번까지 train폴더에 넣고
            9500-10000번까지 test 폴더 넣고
    
    2. 현업에서 CNN을 활용해서 가장 많이 응용하는 부분:
        공장에서 만든 제품에 대한 불량품 분류 --> 품질관리요원
        
    3. 256*256 사이즈의 이미지들을 32*32로 일괄 조정
        D:/data/leafs/images/train
        D:/data/leafs/images/test
            train, test 데이터 둘다 resize            

In [1]:
import  cv2
import  os 
import  numpy  as np

path = "D:/data/leafs/images/train"

file_list = os.listdir(path)
for k in file_list:
    img = cv2.imread(path + '/' + k)
    width, height = img.shape[:2]
    resize_img = cv2.resize(img, (32 , 32), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite('D:/data/leafs/images/train_resize/' + k, resize_img)

In [2]:
import  cv2
import  os 
import  numpy  as np

path = "D:/data/leafs/images/test"

file_list = os.listdir(path)
for k in file_list:
    img = cv2.imread(path + '/' + k)
    width, height = img.shape[:2]
    resize_img = cv2.resize(img, (32 , 32), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite('D:/data/leafs/images/test_resize/' + k, resize_img)

    건강한 이파리 : train 1 ~ 9500        질병 이파리 : train 9501 ~ 19000
                    test 1 ~ 500                       test  501 ~ 1000

    4. 훈련 데이터의 라벨과 테스트 데이터의 라벨을 csv로 생성하시오    

In [3]:
path = 'd:/data/leafs/images/train_label.csv'
file = open(path, 'w')

for _ in range(0, 9500):
    file.write(str(1) + '\n')
    
for _ in range(0, 9500):
    file.write(str(0) + '\n')
file.close()

In [4]:
path = 'd:/data/leafs/images/test_label.csv'
file = open(path, 'w')

for _ in range(0, 500):
    file.write(str(1) + '\n')
    
for _ in range(0, 500):
    file.write(str(0) + '\n')
file.close()

    5. cifar10 데이터를 신경망에 로드하기 위해서 만든 4가지 함수가 있는 loader2.py의 내용을 복사해서 
    loader_leaf.py로 만든 후 안의 내용을 이파리 데이터를 로드할 수 있게끔 수정하시오
        image_load
        label_load
        next_batch
        shuffle_batch
        
    6. 이파리 데이터가 잘 load 되는지 확인하시오

In [1]:
import loader_leaf as ll
train_image = 'd:/data/leafs/images/train_resize/'
test_image = 'd:/data/leafs/images/test_resize/'
train_label = 'd:/data/leafs/images/train_label.csv'
test_label = 'd:/data/leafs/images/test_label.csv'

print(ll.image_load(train_image).shape)
print(ll.image_load(test_image).shape)
print(ll.label_load(train_label).shape)
print(ll.label_load(test_label).shape)

(19000, 32, 32, 3)
(1000, 32, 32, 3)
(19000, 2)
(1000, 2)


### <b>■ 사진을 분류할 수 있는 신경망 구성 환경</b>
    1. 텐서 플로우 1.x 버전으로만 수행하는 방법
    2. 텐서 플로우 1.x 버전(1.14.0) + keras(2.3.1) 로 구현하는 방법
        ---> 구글이 keras 인수
    3. 텐서 플로우 2.x 버전으로 구현하는 방법
    
### <b>■ 이파리 데이터를 분류하는 신경망 설계도의 큰 그림</b>

### ※ 문제142. 우리가 가지고 있는 코드를 활용해서 알